### IMPORTS

In [1]:
import os
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np


### HELPER FUNCTIONS
def get_subdirectories(folder_path):
    subdirectory_path_list = []
    subdirectory_names = os.listdir(folder_path)

    for subdirectories in subdirectory_names:
        full_path = os.path.join(folder_path, subdirectories)
        subdirectory_path_list.append(full_path)
    
    return subdirectory_path_list


def get_image(image_file):
    # Load the image
    image = Image.open(image_file)
    # Convert the image to a NumPy array
    image_array = np.array(image)
    return image_array

def preprocess_dataset(class_files):
    label_list  = []

    for i in range(len(class_files)):
        subdirectories = get_subdirectories(class_files[i])
        subdirectories.sort()

        class_files[i] = subdirectories
        
        for sub in subdirectories: 
            label_list.append(i)

    #Combining the classes together
    concated_list = []
    for classes in class_files:
        concated_list.extend(classes)

    #Shuffling the classes
    concated_list = np.array(concated_list)
    label_list = np.array(label_list)

    # Generate a random permutation of indices
    indices = np.random.permutation(len(concated_list))

    # Apply the permutation to both lists
    shuffled_class_list = concated_list[indices]
    shuffled_label_list = label_list[indices]
    
    shuffled_class_list = [get_image(image_file) for image_file in shuffled_class_list]

    return shuffled_class_list, shuffled_label_list


def print_tree(startpath, level=0):
    """
    Recursively print a directory structure like a tree.
    """
    if level == 0:
        print(f"Root: {startpath}")
    for item in os.listdir(startpath):
        item_path = os.path.join(startpath, item)
        if os.path.isdir(item_path):
            print("   " * level + f"|- {item}")  # Indentation for tree levels
            print_tree(item_path, level + 1)  # Recursively print subdirectories

# Replace this with your dataset's root path
from kagglehub import dataset_download

# Step 1: Download dataset
path = dataset_download("tawsifurrahman/covid19-radiography-database")
print("Dataset downloaded to:", path)

dataset_path = path  # Replace with actual path
print_tree(dataset_path)

file = get_subdirectories(path)
print(file)

files_list = get_subdirectories(file[0])
files_list.sort()
print(np.array(files_list))
#Getting the files from the dataset
normal_file = get_subdirectories(files_list[4])
normal_file.sort()
print(normal_file)

covid_file =  get_subdirectories(files_list[0])
covid_file.sort()
print(covid_file)

normal_images = normal_file[0]
# print(normal_images)

covid_images = covid_file[0]
# print(covid_images)
class_files = [covid_images, normal_images]
class_files, label_list = preprocess_dataset(class_files)


/home/smg0092/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset downloaded to: /home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5
Root: /home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5
|- COVID-19_Radiography_Dataset
   |- Viral Pneumonia
      |- masks
      |- images
   |- COVID
      |- masks
      |- images
   |- Normal
      |- masks
      |- images
   |- Lung_Opacity
      |- masks
      |- images
['/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset']
['/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/COVID'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/COVID.metadata.xlsx'
 '/home/smg0092/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5/COVID-19_Radiography_Dataset/Lung_Opacity'
 '/home/smg

TEST IF GPU WORKING

In [2]:
# import tensorflow as tf

# # List available physical devices
# print("Physical devices:", tf.config.list_physical_devices())

# # Check if TensorFlow is using a GPU
# print("Is TensorFlow using GPU?", tf.test.is_gpu_available())

# # Get the GPU device name
# print("GPU device name:", tf.test.gpu_device_name())


CNN MODEL

In [3]:
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

X, y = np.array(class_files), np.array(label_list)
print(X.shape)
print(y.shape)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42)



# Add a channel dimension: Shape becomes (100, 299, 299, 1)
X = np.expand_dims(X, axis=-1)
# Normalize pixel values to range [0, 1]
X = X / 255.0


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 1)),  # Input shape includes the channel
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # Adjust the output layer for your number of classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()



2024-11-27 11:50:45.791331: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 11:50:45.798568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732729845.806284   13569 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732729845.808463   13569 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 11:50:45.816663: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

(13808, 299, 299)
(13808,)


/home/smg0092/myenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1732729848.862578   13569 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14380 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 297, 297, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 148, 148, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 146, 146, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    20,070,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,164,490 (76.92 MB)

 Trainable params: 20,164,490 (76.92 MB)

 Non-trainable params: 0 (0.00 B)

RUN CNN

In [4]:
model_hist = model.fit(X_train, y_train, epochs=60, validation_data=(X_test, y_test))

from sklearn.metrics import classification_report


score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict(X_test)  # Get predictions for the test data
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels
y_true = y_test  # True labels for the test data
label_dict = {"Covid": 0, "Normal": 1}
report = classification_report(y_true, y_pred_classes, target_names=label_dict)
print(report)

Epoch 1/60


I0000 00:00:1732729850.646204   13726 service.cc:148] XLA service 0x794bbc003a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732729850.646230   13726 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti Laptop GPU, Compute Capability 8.6
2024-11-27 11:50:50.666516: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732729850.750858   13726 cuda_dnn.cc:529] Loaded cuDNN version 90501


  3/303 ━━━━━━━━━━━━━━━━━━━━ 13s 45ms/step - accuracy: 0.3715 - loss: 140.4122

I0000 00:00:1732729857.049801   13726 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


303/303 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7615 - loss: 12.3854

2024-11-27 11:51:12.705288: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_84', 4 bytes spill stores, 4 bytes spill loads



303/303 ━━━━━━━━━━━━━━━━━━━━ 26s 61ms/step - accuracy: 0.7616 - loss: 12.3542 - val_accuracy: 0.8642 - val_loss: 0.3444
Epoch 2/60
303/303 ━━━━━━━━━━━━━━━━━━━━ 15s 50ms/step - accuracy: 0.8535 - loss: 0.3687 - val_accuracy: 0.8878 - val_loss: 0.2719
Epoch 3/60
303/303 ━━━━━━━━━━━━━━━━━━━━ 15s 51ms/step - accuracy: 0.8761 - loss: 0.2900 - val_accuracy: 0.8870 - val_loss: 0.2779
Epoch 4/60
303/303 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.9128 - loss: 0.2204 - val_accuracy: 0.9080 - val_loss: 0.2319
Epoch 5/60
303/303 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step - accuracy: 0.9355 - loss: 0.1562 - val_accuracy: 0.9214 - val_loss: 0.2037
Epoch 6/60
303/303 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step - accuracy: 0.9425 - loss: 0.1395 - val_accuracy: 0.9283 - val_loss: 0.2082
Epoch 7/60
303/303 ━━━━━━━━━━━━━━━━━━━━ 16s 53ms/step - accuracy: 0.9625 - loss: 0.1004 - val_accuracy: 0.9352 - val_loss: 0.2048
Epoch 8/60
303/303 ━━━━━━━━━━━━━━━━━━━━ 16s 53ms/step - accuracy: 0.9683 - loss: 0.0858 - val_accura

VGG Preprocessing

In [7]:

parent_folder = "new_folder"  # Replace with the desired path for the new parent folder

covid_single_images = get_subdirectories(covid_images)
normal_single_images = get_subdirectories(normal_images)

DataSet_size = len(covid_single_images) + len(normal_single_images)

print(DataSet_size)

13808


In [6]:
# import matplotlib.pyplot as plt
# import PIL
# import tensorflow as tf
# import numpy as np
# import os
# from keras.models import Model, Sequential, load_model
# from keras.layers import Dense, Flatten, Dropout
# from keras.applications import VGG16
# from keras.applications.vgg16 import preprocess_input, decode_predictions
# from keras.optimizers import Adam, RMSpropram = X_test[0]
predictions = model.predict(ram)
print("Predicted Class", predictions)

# def vgg(nativeset, data_directory, num_classes, dataset_size, numofruns, epochs, doubledense, finetuning, name):

#     f = open(name[0], "w")
#     plot = open(name[1], "w")
#     acclist_training = []
#     acclist_validation = []
#     acclist_test = []
#     DATASET_SIZE = dataset_size
#     for i in range(numofruns):
#         data_directory = data_directory
#         num_classes = num_classes

#         full_dataset = tf.keras.utils.image_dataset_from_directory(
#             data_directory,
#             color_mode="grayscale",
#             batch_size= None,
#             seed= 42,
#             shuffle=False
#         )

#         full_dataset = full_dataset.shuffle(DATASET_SIZE, seed=42, reshuffle_each_iteration=False)
#         #Change the split
#         train_size = int(0.8 * DATASET_SIZE)
#         val_size = int(0.1 * DATASET_SIZE)
#         test_size = int(0.1 * DATASET_SIZE)
#         training_set = full_dataset.take(train_size)
#         test_set = full_dataset.skip(train_size)
#         validation_set = test_set.skip(val_size)
#         test_set = test_set.take(test_size)


#         print("Train set size: ", len(training_set))
#         print("Test set size: ", len(test_set))
#         print("Valid set size: ", len(validation_set))


#         num_training_examples = 0
#         num_validation_examples = 0
#         for example in training_set:
#             num_training_examples += 1
#         for example in validation_set:
#             num_validation_examples += 1

#         ##############
#         ##############
#         model = VGG16(include_top=True, weights='imagenet')

#         # model.summary()


#         # Select the final layer of the VGG16 Architecture to append our new model for transfer learning
#         transfer_layer = model.get_layer('block5_pool')
#         conv_model = Model(inputs=model.input,
#                         outputs=transfer_layer.output)


#         # Start a new Keras Sequential model.
#         new_model = Sequential()

#         # Add the convolutional part of the VGG16 model from above.
#         new_model.add(conv_model)

#         # Flatten the output of the VGG16 model because it is from a
#         # convolutional layer.
#         new_model.add(Flatten())

#         # Add a dense (aka. fully-connected) layer.
#         # This is for combining features that the VGG16 model has
#         # recognized in the image.
#         new_model.add(Dense(1024, activation='relu'))

#         # Add a dropout-layer which may prevent overfitting and
#         # improve generalization ability to unseen data e.g. the test-set.
#         if (doubledense == True):
#             new_model.add(Dense(1024, activation='relu'))
#         else:
#             new_model.add(Dropout(0.5))

#         # Add the final layer for the actual classification.
#         new_model.add(Dense(num_classes, activation='softmax'))


#         optimizer = Adam(learning_rate=1e-5)

#         loss = 'sparse_categorical_crossentropy'
#         metrics = ['sparse_categorical_accuracy']

#         def print_layer_trainable():
#             for layer in conv_model.layers:
#                 print("{0}:\t{1}".format(layer.trainable, layer.name))

#         if (finetuning == False):
#             #Transfer Learning
#             conv_model.trainable = False

#             for layer in conv_model.layers:
#                 layer.trainable = False

#         print_layer_trainable()

#         new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
#         ##############
#         ##############


#         BATCH_SIZE=16
#         steps_test = len(validation_set) / BATCH_SIZE
#         steps_test

#         epochs = epochs
#         steps_per_epoch = 100

#         IMAGE_RES = 224
        
#         def format_image(image, label):
#             image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
#             return image, label
#         train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
#         train_batches_cm = training_set.map(format_image).batch(BATCH_SIZE).prefetch(1)
#         validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)
#         test_batches = test_set.map(format_image).batch(BATCH_SIZE).prefetch(1)


#         # new_model.load_weights('./checkpoints/ponsvgg16_1000')

#         history = new_model.fit(train_batches,
#                                 epochs=epochs,
#                                 validation_data=validation_batches,
#                                 validation_steps=steps_test)

#         #write the loss and accuracy to a file so that we can graph it later
#         acc = history.history['sparse_categorical_accuracy']
#         loss = history.history['loss']

#         plot.write(str(i + 1) + ":\n")
#         plot.write("loss =  " + str(loss) + "\n")
#         plot.write("acc =  " + str(acc) + "\n")

#         val_acc = history.history['val_sparse_categorical_accuracy']
#         val_loss = history.history['val_loss']
#         plot.write("val_loss =  " + str(val_loss) + "\n")
#         plot.write("val_acc =  " + str(val_acc) + "\n\n\n")
#         plot.flush()

#         result_train = new_model.evaluate(train_batches)
#         result_val = new_model.evaluate(validation_batches)
#         result_test = new_model.evaluate(test_batches)

#         f.write(str(i + 1) + ":\n")
#         f.write("Training classification accuracy: {0:.2%}".format(result_train[1]) + "\n")
#         f.write("Validation classification accuracy: {0:.2%}".format(result_val[1]) + "\n")
#         f.write("Test classification accuracy: {0:.2%}".format(result_test[1]) + "\n\n\n")
#         f.flush()

#         acclist_training.append(result_train[1])
#         acclist_validation.append(result_val[1])
#         acclist_test.append(result_test[1])

#     f.write("Training accuracy: " + str(max(acclist_training)) + " (" + str(sum(acclist_training)/len(acclist_training)) + ")\n")
#     f.write("Validation accuracy: " + str(max(acclist_validation)) + " (" + str(sum(acclist_validation)/len(acclist_validation)) + ")\n")
#     f.write("Test accuracy: " + str(max(acclist_test)) + " (" + str(sum(acclist_test)/len(acclist_test)) + ")\n")
#     f.flush()


# data_directory = parent_folder
# vgg(nativeset=False, 
#     data_directory= data_directory, 
#     num_classes= 2, 
#     dataset_size= DataSet_size, 
#     numofruns=25, 
#     epochs=60, 
#     doubledense=False, 
#     finetuning=True, 
#     name= "Normal_covid_images")


# # #set to pons uc merced
# # DATASET_SIZE = 2056
# # data_directory = ponsdata_directory
# # # vgg(False, data_directory, num_classes, DATASET_SIZE, 1, 60, False, True, name)
# # # vgg(False, data_directory, num_classes, DATASET_SIZE, 1, 60, True, True, name1)
# # vgg(False, data_directory, num_classes, DATASET_SIZE, 1, 400, False, False, name5)
# # vgg(False, data_directory, num_classes, DATASET_SIZE, 1, 400, True, False, name3)

NameError: name 'ram' is not defined